In [1]:
import folium
from folium.plugins import BeautifyIcon
import shapely
import pandas as pd
import openrouteservice as ors
from IPython.core.interactiveshell import InteractiveShell

In [2]:
# First define the map centered around Beira
m = folium.Map(location=[43.640565, -116.339269], tiles='cartodbpositron', zoom_start=13)    

# Next load the delivery locations from CSV file at ../resources/data/idai_health_sites.csv
# ID, Lat, Lon, Open_From, Open_To, Needed_Amount
deliveries_data = pd.read_csv(
    'data/demo_data.csv',
    index_col="ID",
    parse_dates=["Open_From", "Open_To"]
)
# Plot the locations on the map with more info in the ToolTip
for location in deliveries_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))
    
    folium.Marker(
        location=[location.Lat, location.Lon],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)    
    
# The vehicles are all located Centennial High School
depot = [43.649839, -116.336241]

folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="bus", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)
m

In [4]:
# Define the vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx, 
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[17],
            time_window=[1553241600, 1553245200]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    
# Next define the delivery stations
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
deliveries = list()
for delivery in deliveries_data.itertuples():
    deliveries.append(
        ors.optimization.Job(
            id=delivery.Index,
            location=[delivery.Lon, delivery.Lat],
            service=15,  # Assume 30 seconds at each site
            amount=[delivery.Needed_Amount],
            time_windows=[[
                int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(delivery.Open_To.timestamp())
            ]]
        )
    )

In [5]:
# Initialize a client and make the request
ors_client = ors.Client(key='5b3ce3597851110001cf6248a4a8f689bb5f4ee5865a9e838a21f443')  # Get an API key from https://openrouteservice.org/dev/#/signup
result = ors_client.optimization(
    jobs=deliveries,
    vehicles=vehicles,
    geometry=True
)

# Add the output to the map
for color, route in zip(['green', 'red', 'blue'], result['routes']):
    decoded=ors.convert.decode_polyline(route['geometry'])    # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature", 
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                        }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)

folium.LayerControl().add_to(m)
m

In [6]:
# Only extract relevant fields from the response
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,amount,distance,duration
vehicle,,,
0,[17],21295,2923
1,[16],24091,3344
2,[16],20442,3303


In [7]:
# Create a list to display the schedule for all vehicles
stations = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Station ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time
                
            ]
        )
    stations.append(vehicle)

In [9]:
    df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
df_stations_0

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,6,2019-03-22 08:01:46,2019-03-22 08:02:11
2,35,2019-03-22 08:03:12,2019-03-22 08:03:37
3,32,2019-03-22 08:07:02,2019-03-22 08:07:27
4,44,2019-03-22 08:14:33,2019-03-22 08:14:58
5,25,2019-03-22 08:20:25,2019-03-22 08:20:50
6,36,2019-03-22 08:23:26,2019-03-22 08:23:51
7,39,2019-03-22 08:26:16,2019-03-22 08:26:41
8,45,2019-03-22 08:30:39,2019-03-22 08:31:04
9,43,2019-03-22 08:31:56,2019-03-22 08:32:21


In [10]:
df_stations_1 = pd.DataFrame(stations[1], columns=["Station ID", "Arrival", "Departure"])
df_stations_1['Arrival'] = pd.to_datetime(df_stations_1['Arrival'], unit='s')
df_stations_1['Departure'] = pd.to_datetime(df_stations_1['Departure'], unit='s')
df_stations_1

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,20,2019-03-22 08:01:05,2019-03-22 08:01:30
2,18,2019-03-22 08:04:13,2019-03-22 08:04:38
3,28,2019-03-22 08:06:50,2019-03-22 08:07:15
4,9,2019-03-22 08:09:57,2019-03-22 08:10:22
5,5,2019-03-22 08:15:44,2019-03-22 08:16:09
6,10,2019-03-22 08:18:25,2019-03-22 08:18:50
7,17,2019-03-22 08:21:49,2019-03-22 08:22:14
8,46,2019-03-22 08:25:05,2019-03-22 08:25:30
9,1,2019-03-22 08:29:10,2019-03-22 08:29:35


In [13]:
df_stations_2 = pd.DataFrame(stations[2], columns=["Station ID", "Arrival", "Departure"])
df_stations_2['Arrival'] = pd.to_datetime(df_stations_2['Arrival'], unit='s')
df_stations_2['Departure'] = pd.to_datetime(df_stations_2['Departure'], unit='s')
df_stations_2

,Station ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,14,2019-03-22 08:06:04,2019-03-22 08:06:29
2,16,2019-03-22 08:09:51,2019-03-22 08:10:16
3,31,2019-03-22 08:13:38,2019-03-22 08:14:03
4,22,2019-03-22 08:16:28,2019-03-22 08:16:53
5,33,2019-03-22 08:19:10,2019-03-22 08:19:35
6,38,2019-03-22 08:23:58,2019-03-22 08:24:23
7,12,2019-03-22 08:27:42,2019-03-22 08:28:07
8,15,2019-03-22 08:31:04,2019-03-22 08:31:29
9,49,2019-03-22 08:31:55,2019-03-22 08:32:20


In [7]:
m